# Covid-19 ... noch mehr Diagramme
Usprünglich nach einer Idee und einem Script von [Tiba Razmi](https://www.linkedin.com/pulse/covid-19-swiss-model-prediction-tiba-razmi/) mit Daten aus [Kaggle](https://www.kaggle.com/imdevskp/corona-virus-report).  
Covid-19 Daten vom [European Centre for Disease Prevention and Control](https://www.ecdc.europa.eu/en/publications-data/download-todays-data-geographic-distribution-covid-19-cases-worldwide)  
Covid-19 Daten für die Schweiz [GitHub cases (daenuprobst)](https://github.com/daenuprobst/covid19-cases-switzerland/blob/master/covid19_cases_switzerland_openzh.csv), [GitHub fatalities (daenuprobst)](https://github.com/daenuprobst/covid19-cases-switzerland/blob/master/covid19_fatalities_switzerland_openzh.csv), siehe auch: [wikipedia](https://de.wikipedia.org/wiki/COVID-19-Pandemie_in_der_Schweiz),   
Daten für die Lombardei [GitHub (Umberto Rosini)](https://github.com/pcm-dpc/COVID-19/tree/master/dati-regioni), siehe auch: [wikipedia](https://de.wikipedia.org/wiki/COVID-19-Pandemie_in_Italien), [Il Messaggero](https://utils.cedsdigital.it/coronavirus/) (link aus [ilmessaggero.it](https://ilmessaggero.it))   
Bevölkerungsdaten der Länder aus [Worldometers.info](https://www.worldometers.info/world-population/population-by-country/)  
weitere Quellen: [worldometers](https://www.worldometers.info/coronavirus/)  
Bevölkerungsdaten Tessin und Lombardei aus Wikipedia.    

In [5]:
'''md
## Daten importieren und vorbereiten
'''
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# hide warnings
import warnings
warnings.filterwarnings('ignore')
''' md
## Daten aus ECDC einlesen
'''
df = pd.read_csv('ecdc_data.csv', delimiter=',')
        #nRow, nCol = df.shape
        #print(f'There are {nRow} rows and {nCol} columns')
        
# format date column
df.dateRep = pd.to_datetime(df.dateRep, format='%d/%m/%Y')

# retain only relevant columns
df2 = df[["dateRep","cases","deaths","countriesAndTerritories", "popData2018"]]

#rename columns to match variables used in plotly
df2.columns=["Date","cases","deaths","Country", "popData2018"]

#select countries
       #what countries are there?
       #df["countriesAndTerritories"].unique()

c_s = ["Lombardia","Ticino","Italy","Spain","France","United_Kingdom","Switzerland","Sweden","United_States_of_America","Germany","South_Korea"]
df_selec = df2[df2["Country"].isin(c_s)]

# calculation of cumulative sum from ECDC data                    cumsum
df_selec=df_selec.sort_values(by=['Country','Date'])
df_selec['cases_pday'] = df_selec.cases
df_selec['cases_pday_rolling']=df_selec['cases_pday'].rolling(window=3).mean()
df_selec['cases'] = df_selec.groupby('Country')['cases'].transform(pd.Series.cumsum)
df_selec['deaths_pday'] = df_selec.deaths
df_selec['deaths_pday_rolling']=df_selec.groupby('Country')['deaths_pday'].rolling(window=3).mean().reset_index(0,drop=True)
df_selec['deaths'] = df_selec.groupby('Country')['deaths'].transform(pd.Series.cumsum)

#calculation of additional parameters
df_selec['Mortality_%'] = df_selec.deaths / df_selec.cases * 100
df_selec['Extent_%'] = df_selec.cases / df_selec.popData2018 * 100
df_selec['Deaths%pop']=df_selec.deaths / df_selec.popData2018 * 100
df_selec['deaths_pday_rolling%']=df_selec.deaths_pday_rolling / df_selec.popData2018 * 100
'''md
# Plotly
'''

# Quelle: https://www.youtube.com/watch?v=XUNaGFa9xCM
# dort gibt es ein weiteres Beispiel mit Widgets und interactive
#df.loc[df['Country'] == "Italy"]

import plotly as py
import plotly.graph_objs as go
import ipywidgets as widgets
import numpy as np
py.offline.init_notebook_mode(connected=True)
#                                                                                   Fig. (a)
layouta = go.Layout(
    title='Case fatality rate (deaths in % of confirmed cases)',
    yaxis=dict(
        title='% of confirmed cases'
    ),
)

trace0a = go.Scatter(
    x=df_selec.loc[df_selec['Country'] == c_s[0],"Date"],
    y=df_selec.loc[df_selec['Country'] == c_s[0], "Mortality_%"],
    mode='lines',
    name=c_s[0],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace1a = go.Scatter(
    x=df_selec.loc[df_selec['Country'] == c_s[1],"Date"],
    y=df_selec.loc[df_selec['Country'] == c_s[1],"Mortality_%"],
    mode='lines',
    name=c_s[1],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace2a = go.Scatter(
    x=df_selec.loc[df_selec['Country'] == c_s[2],"Date"],
    y=df_selec.loc[df_selec['Country'] == c_s[2], "Mortality_%"],
    mode='lines',
    name=c_s[2],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace3a = go.Scatter(
    x=df_selec.loc[df_selec['Country'] == c_s[3],"Date"],
    y=df_selec.loc[df_selec['Country'] == c_s[3],"Mortality_%"],
    mode='lines',
    name=c_s[3],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace4a = go.Scatter(
    x=df_selec.loc[df_selec['Country'] == c_s[4],"Date"],
    y=df_selec.loc[df_selec['Country'] == c_s[4], "Mortality_%"],
    mode='lines',
    name=c_s[4],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace5a = go.Scatter(
    x=df_selec.loc[df_selec['Country'] == c_s[5],"Date"],
    y=df_selec.loc[df_selec['Country'] == c_s[5],"Mortality_%"],
    mode='lines',
    name=c_s[5],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace6a = go.Scatter(
    x=df_selec.loc[df_selec['Country'] == c_s[6],"Date"],
    y=df_selec.loc[df_selec['Country'] == c_s[6],"Mortality_%"],
    mode='lines',
    name=c_s[6],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace7a = go.Scatter(
    x=df_selec.loc[df_selec['Country'] == c_s[7],"Date"],
    y=df_selec.loc[df_selec['Country'] == c_s[7],"Mortality_%"],
    mode='lines',
    name=c_s[7],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace8a = go.Scatter(
    x=df_selec.loc[df_selec['Country'] == c_s[8],"Date"],
    y=df_selec.loc[df_selec['Country'] == c_s[8],"Mortality_%"],
    mode='lines',
    name=c_s[8],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace9a = go.Scatter(
    x=df_selec.loc[df_selec['Country'] == c_s[9],"Date"],
    y=df_selec.loc[df_selec['Country'] == c_s[9],"Mortality_%"],
    mode='lines',
    name=c_s[9],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace10a = go.Scatter(
    x=df_selec.loc[df_selec['Country'] == c_s[10],"Date"],
    y=df_selec.loc[df_selec['Country'] == c_s[10],"Mortality_%"],
    mode='lines',
    name=c_s[10],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
#                                                                                   Fig. (aa)
layoutaa = go.Layout(
    title='Daily deaths rate (% of population, 3 day moving average)',
    yaxis=dict(
        title='% of population'
    ),
)

trace0aa = go.Scatter(
    x=df_selec.loc[df_selec['Country'] == c_s[0],"Date"],
    y=df_selec.loc[df_selec['Country'] == c_s[0], "deaths_pday_rolling%"],
    mode='lines',
    name=c_s[0],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace1aa = go.Scatter(
    x=df_selec.loc[df_selec['Country'] == c_s[1],"Date"],
    y=df_selec.loc[df_selec['Country'] == c_s[1],"deaths_pday_rolling%"],
    mode='lines',
    name=c_s[1],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace2aa = go.Scatter(
    x=df_selec.loc[df_selec['Country'] == c_s[2],"Date"],
    y=df_selec.loc[df_selec['Country'] == c_s[2], "deaths_pday_rolling%"],
    mode='lines',
    name=c_s[2],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace3aa = go.Scatter(
    x=df_selec.loc[df_selec['Country'] == c_s[3],"Date"],
    y=df_selec.loc[df_selec['Country'] == c_s[3],"deaths_pday_rolling%"],
    mode='lines',
    name=c_s[3],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace4aa = go.Scatter(
    x=df_selec.loc[df_selec['Country'] == c_s[4],"Date"],
    y=df_selec.loc[df_selec['Country'] == c_s[4], "deaths_pday_rolling%"],
    mode='lines',
    name=c_s[4],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace5aa = go.Scatter(
    x=df_selec.loc[df_selec['Country'] == c_s[5],"Date"],
    y=df_selec.loc[df_selec['Country'] == c_s[5],"deaths_pday_rolling%"],
    mode='lines',
    name=c_s[5],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace6aa = go.Scatter(
    x=df_selec.loc[df_selec['Country'] == c_s[6],"Date"],
    y=df_selec.loc[df_selec['Country'] == c_s[6],"deaths_pday_rolling%"],
    mode='lines',
    name=c_s[6],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace7aa = go.Scatter(
    x=df_selec.loc[df_selec['Country'] == c_s[7],"Date"],
    y=df_selec.loc[df_selec['Country'] == c_s[7],"deaths_pday_rolling%"],
    mode='lines',
    name=c_s[7],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace8aa = go.Scatter(
    x=df_selec.loc[df_selec['Country'] == c_s[8],"Date"],
    y=df_selec.loc[df_selec['Country'] == c_s[8],"deaths_pday_rolling%"],
    mode='lines',
    name=c_s[8],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace9aa = go.Scatter(
    x=df_selec.loc[df_selec['Country'] == c_s[9],"Date"],
    y=df_selec.loc[df_selec['Country'] == c_s[9],"deaths_pday_rolling%"],
    mode='lines',
    name=c_s[9],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace10aa = go.Scatter(
    x=df_selec.loc[df_selec['Country'] == c_s[10],"Date"],
    y=df_selec.loc[df_selec['Country'] == c_s[10],"deaths_pday_rolling%"],
    mode='lines',
    name=c_s[10],
    line=dict(
        shape='spline',
        dash='dot'
    )
)#                                                                                   Fig. (b)
'''layoutb = go.Layout(
    title='Per capita confirmed cases (confirmed cases in % of population)',
    yaxis=dict(
        title='% of population'
    ),
)

trace0b = go.Scatter(
    x=df_selec.loc[df_selec['Country'] == c_s[0],"Date"],
    y=df_selec.loc[df_selec['Country'] == c_s[0], "Extent_%"],
    mode='lines',
    name=c_s[0],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace1b = go.Scatter(
    x=df_selec.loc[df_selec['Country'] == c_s[1],"Date"],
    y=df_selec.loc[df_selec['Country'] == c_s[1],"Extent_%"],
    mode='lines',
    name=c_s[1],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace2b = go.Scatter(
    x=df_selec.loc[df_selec['Country'] == c_s[2],"Date"],
    y=df_selec.loc[df_selec['Country'] == c_s[2], "Extent_%"],
    mode='lines',
    name=c_s[2],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace3b = go.Scatter(
    x=df_selec.loc[df_selec['Country'] == c_s[3],"Date"],
    y=df_selec.loc[df_selec['Country'] == c_s[3],"Extent_%"],
    mode='lines',
    name=c_s[3],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace4b = go.Scatter(
    x=df_selec.loc[df_selec['Country'] == c_s[4],"Date"],
    y=df_selec.loc[df_selec['Country'] == c_s[4], "Extent_%"],
    mode='lines',
    name=c_s[4],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace5b = go.Scatter(
    x=df_selec.loc[df_selec['Country'] == c_s[5],"Date"],
    y=df_selec.loc[df_selec['Country'] == c_s[5],"Extent_%"],
    mode='lines',
    name=c_s[5],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace6b = go.Scatter(
    x=df_selec.loc[df_selec['Country'] == c_s[6],"Date"],
    y=df_selec.loc[df_selec['Country'] == c_s[6],"Extent_%"],
    mode='lines',
    name=c_s[6],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace7b = go.Scatter(
    x=df_selec.loc[df_selec['Country'] == c_s[7],"Date"],
    y=df_selec.loc[df_selec['Country'] == c_s[7],"Extent_%"],
    mode='lines',
    name=c_s[7],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace8b = go.Scatter(
    x=df_selec.loc[df_selec['Country'] == c_s[8],"Date"],
    y=df_selec.loc[df_selec['Country'] == c_s[8],"Extent_%"],
    mode='lines',
    name=c_s[8],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace9b = go.Scatter(
    x=df_selec.loc[df_selec['Country'] == c_s[9],"Date"],
    y=df_selec.loc[df_selec['Country'] == c_s[9],"Extent_%"],
    mode='lines',
    name=c_s[9],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace10b = go.Scatter(
    x=df_selec.loc[df_selec['Country'] == c_s[10],"Date"],
    y=df_selec.loc[df_selec['Country'] == c_s[10],"Extent_%"],
    mode='lines',
    name=c_s[10],
    line=dict(
        shape='spline',
        dash='dot'
    )
)

#                                                                                   Fig. (c)
'''
layoutc = go.Layout(
    title='Crude mortality rate (deaths in % of population)',
    yaxis=dict(
        title='% of population'
    ),
)

trace0c = go.Scatter(
    x=df_selec.loc[df_selec['Country'] == c_s[0],"Date"],
    y=df_selec.loc[df_selec['Country'] == c_s[0], "Deaths%pop"],
    mode='lines',
    name=c_s[0],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace1c = go.Scatter(
    x=df_selec.loc[df_selec['Country'] == c_s[1],"Date"],
    y=df_selec.loc[df_selec['Country'] == c_s[1],"Deaths%pop"],
    mode='lines',
    name=c_s[1],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace2c = go.Scatter(
    x=df_selec.loc[df_selec['Country'] == c_s[2],"Date"],
    y=df_selec.loc[df_selec['Country'] == c_s[2], "Deaths%pop"],
    mode='lines',
    name=c_s[2],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace3c = go.Scatter(
    x=df_selec.loc[df_selec['Country'] == c_s[3],"Date"],
    y=df_selec.loc[df_selec['Country'] == c_s[3],"Deaths%pop"],
    mode='lines',
    name=c_s[3],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace4c = go.Scatter(
    x=df_selec.loc[df_selec['Country'] == c_s[4],"Date"],
    y=df_selec.loc[df_selec['Country'] == c_s[4], "Deaths%pop"],
    mode='lines',
    name=c_s[4],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace5c = go.Scatter(
    x=df_selec.loc[df_selec['Country'] == c_s[5],"Date"],
    y=df_selec.loc[df_selec['Country'] == c_s[5],"Deaths%pop"],
    mode='lines',
    name=c_s[5],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace6c = go.Scatter(
    x=df_selec.loc[df_selec['Country'] == c_s[6],"Date"],
    y=df_selec.loc[df_selec['Country'] == c_s[6],"Deaths%pop"],
    mode='lines',
    name=c_s[6],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace7c = go.Scatter(
    x=df_selec.loc[df_selec['Country'] == c_s[7],"Date"],
    y=df_selec.loc[df_selec['Country'] == c_s[7],"Deaths%pop"],
    mode='lines',
    name=c_s[7],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace8c = go.Scatter(
    x=df_selec.loc[df_selec['Country'] == c_s[8],"Date"],
    y=df_selec.loc[df_selec['Country'] == c_s[8],"Deaths%pop"],
    mode='lines',
    name=c_s[8],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace9c = go.Scatter(
    x=df_selec.loc[df_selec['Country'] == c_s[9],"Date"],
    y=df_selec.loc[df_selec['Country'] == c_s[9],"Deaths%pop"],
    mode='lines',
    name=c_s[9],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace10c = go.Scatter(
    x=df_selec.loc[df_selec['Country'] == c_s[10],"Date"],
    y=df_selec.loc[df_selec['Country'] == c_s[10],"Deaths%pop"],
    mode='lines',
    name=c_s[10],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
figc = go.Figure(data=[trace0c, trace1c, trace2c, trace3c, trace4c, trace5c, trace6c, trace7c, trace8c, trace9c, trace10c], layout=layoutc)
py.offline.iplot(figc)

figaa = go.Figure(data=[trace0aa, trace1aa, trace2aa, trace3aa, trace4aa, trace5aa, trace6aa, trace7aa, trace8aa, trace9aa, trace10aa], layout=layoutaa)
py.offline.iplot(figaa)
'''
figb = go.Figure(data=[trace0b, trace1b, trace2b, trace3b, trace4b, trace5b, trace6b, trace7b, trace8b, trace9b, trace10b], layout=layoutb)
py.offline.iplot(figb)
'''
figa = go.Figure(data=[trace0a, trace1a, trace2a, trace3a, trace4a, trace5a, trace6a, trace7a, trace8a, trace9a, trace10a], layout=layouta)
py.offline.iplot(figa)





-----------------------------------------
# Wöchentliche Todesfälle in der Schweiz 2020
### Grafik des BFS ergänzt mit den bestätigten Todesfällen durch COVID-19 (rote Balken):

<img src="WTF2020.png" alt="Drawing" style="width: 800px;"/>

Quelle: [BFS, 2020](https://www.bfs.admin.ch/bfs/de/home/statistiken/gesundheit/gesundheitszustand/sterblichkeit-todesursachen.html)

-----------------------------------------
### Zum Vergleich die Grippewelle von 2016/2017:

<img src="WTF1617.png" alt="Drawing" style="width: 800px;"/>



Quelle: [BFS, 2017](https://www.bfs.admin.ch/bfs/de/home/statistiken/gesundheit/gesundheitszustand/sterblichkeit-todesursachen.assetdetail.4482306.html)

-----------------------------------------

In [2]:
#select countries
       #what countries are there?
       #df["countriesAndTerritories"].unique()

cantons = ["TI","VD","GE","VS","BS","BE","ZH", "GR", "CH","FR"]
df_cantons = df2[df2["Country"].isin(cantons)]
df_cantons=df_cantons.sort_values(by=['Country','Date'])

#calculation of additional parameters
df_cantons['Mortality_%'] = df_cantons.deaths / df_cantons.cases * 100
df_cantons['Extent_%'] = df_cantons.cases / df_cantons.popData2018 * 100
df_cantons['Deaths%pop']=df_cantons.deaths / df_cantons.popData2018 * 100

'''md
# Plotly
'''

# Quelle: https://www.youtube.com/watch?v=XUNaGFa9xCM
# dort gibt es ein weiteres Beispiel mit Widgets und interactive
#df.loc[df['Country'] == "Italy"]

import plotly as py
import plotly.graph_objs as go
import ipywidgets as widgets
import numpy as np
py.offline.init_notebook_mode(connected=True)
#                                                                                   Fig. (a)
layouta = go.Layout(
    title='Case fatality rate (deaths in % of confirmed cases)',
    yaxis=dict(
        title='% of confirmed cases'
    ),
)

trace_cantons0a = go.Scatter(
    x=df_cantons.loc[df_cantons['Country'] == cantons[0],"Date"],
    y=df_cantons.loc[df_cantons['Country'] == cantons[0], "Mortality_%"],
    mode='lines',
    name=cantons[0],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace_cantons1a = go.Scatter(
    x=df_cantons.loc[df_cantons['Country'] == cantons[1],"Date"],
    y=df_cantons.loc[df_cantons['Country'] == cantons[1],"Mortality_%"],
    mode='lines',
    name=cantons[1],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace_cantons2a = go.Scatter(
    x=df_cantons.loc[df_cantons['Country'] == cantons[2],"Date"],
    y=df_cantons.loc[df_cantons['Country'] == cantons[2], "Mortality_%"],
    mode='lines',
    name=cantons[2],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace_cantons3a = go.Scatter(
    x=df_cantons.loc[df_cantons['Country'] == cantons[3],"Date"],
    y=df_cantons.loc[df_cantons['Country'] == cantons[3],"Mortality_%"],
    mode='lines',
    name=cantons[3],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace_cantons4a = go.Scatter(
    x=df_cantons.loc[df_cantons['Country'] == cantons[4],"Date"],
    y=df_cantons.loc[df_cantons['Country'] == cantons[4], "Mortality_%"],
    mode='lines',
    name=cantons[4],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace_cantons5a = go.Scatter(
    x=df_cantons.loc[df_cantons['Country'] == cantons[5],"Date"],
    y=df_cantons.loc[df_cantons['Country'] == cantons[5],"Mortality_%"],
    mode='lines',
    name=cantons[5],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace_cantons6a = go.Scatter(
    x=df_cantons.loc[df_cantons['Country'] == cantons[6],"Date"],
    y=df_cantons.loc[df_cantons['Country'] == cantons[6],"Mortality_%"],
    mode='lines',
    name=cantons[6],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace_cantons7a = go.Scatter(
    x=df_cantons.loc[df_cantons['Country'] == cantons[7],"Date"],
    y=df_cantons.loc[df_cantons['Country'] == cantons[7],"Mortality_%"],
    mode='lines',
    name=cantons[7],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace_cantons8a = go.Scatter(
    x=df_cantons.loc[df_cantons['Country'] == cantons[8],"Date"],
    y=df_cantons.loc[df_cantons['Country'] == cantons[8],"Mortality_%"],
    mode='lines',
    name=cantons[8],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace_cantons9a = go.Scatter(
    x=df_cantons.loc[df_cantons['Country'] == cantons[9],"Date"],
    y=df_cantons.loc[df_cantons['Country'] == cantons[9],"Mortality_%"],
    mode='lines',
    name=cantons[9],
    line=dict(
        shape='spline',
        dash='dot'
    )
)

#                                                                                   Fig. (b)
layoutb = go.Layout(
    title='Per capita confirmed cases (confirmed cases in % of population)',
    yaxis=dict(
        title='% of population'
    ),
)

trace_cantons0b = go.Scatter(
    x=df_cantons.loc[df_cantons['Country'] == cantons[0],"Date"],
    y=df_cantons.loc[df_cantons['Country'] == cantons[0], "Extent_%"],
    mode='lines',
    name=cantons[0],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace_cantons1b = go.Scatter(
    x=df_cantons.loc[df_cantons['Country'] == cantons[1],"Date"],
    y=df_cantons.loc[df_cantons['Country'] == cantons[1],"Extent_%"],
    mode='lines',
    name=cantons[1],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace_cantons2b = go.Scatter(
    x=df_cantons.loc[df_cantons['Country'] == cantons[2],"Date"],
    y=df_cantons.loc[df_cantons['Country'] == cantons[2], "Extent_%"],
    mode='lines',
    name=cantons[2],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace_cantons3b = go.Scatter(
    x=df_cantons.loc[df_cantons['Country'] == cantons[3],"Date"],
    y=df_cantons.loc[df_cantons['Country'] == cantons[3],"Extent_%"],
    mode='lines',
    name=cantons[3],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace_cantons4b = go.Scatter(
    x=df_cantons.loc[df_cantons['Country'] == cantons[4],"Date"],
    y=df_cantons.loc[df_cantons['Country'] == cantons[4], "Extent_%"],
    mode='lines',
    name=cantons[4],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace_cantons5b = go.Scatter(
    x=df_cantons.loc[df_cantons['Country'] == cantons[5],"Date"],
    y=df_cantons.loc[df_cantons['Country'] == cantons[5],"Extent_%"],
    mode='lines',
    name=cantons[5],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace_cantons6b = go.Scatter(
    x=df_cantons.loc[df_cantons['Country'] == cantons[6],"Date"],
    y=df_cantons.loc[df_cantons['Country'] == cantons[6],"Extent_%"],
    mode='lines',
    name=cantons[6],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace_cantons7b = go.Scatter(
    x=df_cantons.loc[df_cantons['Country'] == cantons[7],"Date"],
    y=df_cantons.loc[df_cantons['Country'] == cantons[7],"Extent_%"],
    mode='lines',
    name=cantons[7],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace_cantons8b = go.Scatter(
    x=df_cantons.loc[df_cantons['Country'] == cantons[8],"Date"],
    y=df_cantons.loc[df_cantons['Country'] == cantons[8],"Extent_%"],
    mode='lines',
    name=cantons[8],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace_cantons9b = go.Scatter(
    x=df_cantons.loc[df_cantons['Country'] == cantons[9],"Date"],
    y=df_cantons.loc[df_cantons['Country'] == cantons[9],"Extent_%"],
    mode='lines',
    name=cantons[9],
    line=dict(
        shape='spline',
        dash='dot'
    )
)

#                                                                                   Fig. (c)

layoutc = go.Layout(
    title='Crude mortality rate (deaths in % of population)',
    yaxis=dict(
        title='% of population'
    ),
)

trace_cantons0c = go.Scatter(
    x=df_cantons.loc[df_cantons['Country'] == cantons[0],"Date"],
    y=df_cantons.loc[df_cantons['Country'] == cantons[0], "Deaths%pop"],
    mode='lines',
    name=cantons[0],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace_cantons1c = go.Scatter(
    x=df_cantons.loc[df_cantons['Country'] == cantons[1],"Date"],
    y=df_cantons.loc[df_cantons['Country'] == cantons[1],"Deaths%pop"],
    mode='lines',
    name=cantons[1],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace_cantons2c = go.Scatter(
    x=df_cantons.loc[df_cantons['Country'] == cantons[2],"Date"],
    y=df_cantons.loc[df_cantons['Country'] == cantons[2], "Deaths%pop"],
    mode='lines',
    name=cantons[2],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace_cantons3c = go.Scatter(
    x=df_cantons.loc[df_cantons['Country'] == cantons[3],"Date"],
    y=df_cantons.loc[df_cantons['Country'] == cantons[3],"Deaths%pop"],
    mode='lines',
    name=cantons[3],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace_cantons4c = go.Scatter(
    x=df_cantons.loc[df_cantons['Country'] == cantons[4],"Date"],
    y=df_cantons.loc[df_cantons['Country'] == cantons[4], "Deaths%pop"],
    mode='lines',
    name=cantons[4],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace_cantons5c = go.Scatter(
    x=df_cantons.loc[df_cantons['Country'] == cantons[5],"Date"],
    y=df_cantons.loc[df_cantons['Country'] == cantons[5],"Deaths%pop"],
    mode='lines',
    name=cantons[5],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace_cantons6c = go.Scatter(
    x=df_cantons.loc[df_cantons['Country'] == cantons[6],"Date"],
    y=df_cantons.loc[df_cantons['Country'] == cantons[6],"Deaths%pop"],
    mode='lines',
    name=cantons[6],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace_cantons7c = go.Scatter(
    x=df_cantons.loc[df_cantons['Country'] == cantons[7],"Date"],
    y=df_cantons.loc[df_cantons['Country'] == cantons[7],"Deaths%pop"],
    mode='lines',
    name=cantons[7],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace_cantons8c = go.Scatter(
    x=df_cantons.loc[df_cantons['Country'] == cantons[8],"Date"],
    y=df_cantons.loc[df_cantons['Country'] == cantons[8],"Deaths%pop"],
    mode='lines',
    name=cantons[8],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
trace_cantons9c = go.Scatter(
    x=df_cantons.loc[df_cantons['Country'] == cantons[9],"Date"],
    y=df_cantons.loc[df_cantons['Country'] == cantons[9],"Deaths%pop"],
    mode='lines',
    name=cantons[9],
    line=dict(
        shape='spline',
        dash='dot'
    )
)
fig_cantonsc = go.Figure(data=[trace_cantons0c, trace_cantons1c, trace_cantons2c, trace_cantons3c, trace_cantons4c, trace_cantons5c, trace_cantons6c, trace_cantons7c, trace_cantons8c, trace_cantons9c], layout=layoutc)
py.offline.iplot(fig_cantonsc)

fig_cantonsb = go.Figure(data=[trace_cantons0b, trace_cantons1b, trace_cantons2b, trace_cantons3b, trace_cantons4b, trace_cantons5b, trace_cantons6b, trace_cantons7b, trace_cantons8b, trace_cantons9b], layout=layoutb)
py.offline.iplot(fig_cantonsb)

fig_cantonsa = go.Figure(data=[trace_cantons0a, trace_cantons1a, trace_cantons2a, trace_cantons3a, trace_cantons4a, trace_cantons5a, trace_cantons6a, trace_cantons7a, trace_cantons8a, trace_cantons9a], layout=layouta)
py.offline.iplot(fig_cantonsa)